In [27]:
from loguru import logger
import ndf.download
from ndf.datamining import datamining
import warnings
import sys
import pandas as pd

warnings.simplefilter(action='ignore', category=FutureWarning)

log_level = 'ERROR'
logger.add(sys.stderr, format="{time} {level} {message}", filter="my_module", level=log_level)
# logger.add(sys.stdout, colorize=True, format="<green>{time}</green> <level>{message}</level>")
logger.add("ndfdownload.log", rotation="30 MB", level=log_level)

6

In [28]:
import requests
import time

MAX_RETRIES = 4
TIME_TO_WAIT = 10

def download_manager(url):
    r = None
    # logger.info(f'Download settings: MAX_RETRIES:{MAX_RETRIES} | TIME_TO_WAIT:{TIME_TO_WAIT}')
    for _ in range(MAX_RETRIES):
        try:
            r = requests.get(url, verify=False)
        except requests.exceptions.RequestException as e:
            print(f'Download fail, {e}')
        if r:
            print(f'Download Ok')
            break
        else:
            wait = TIME_TO_WAIT * (_ + 1) if _ < 3 else TIME_TO_WAIT * (_ * 3)
            print(f'Download fail, retry in {wait} seconds...')
            time.sleep(wait)
    return r

In [29]:
download_manager('https://www.tullettprebon.com/swap-execution-facility/daily-activity-summary.aspxa')

/Users/cleiton/PycharmProjects/ndfutil/venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tullettprebon.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Download fail, retry in 10 seconds...


/Users/cleiton/PycharmProjects/ndfutil/venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tullettprebon.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Download fail, retry in 20 seconds...


/Users/cleiton/PycharmProjects/ndfutil/venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tullettprebon.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Download fail, retry in 30 seconds...


/Users/cleiton/PycharmProjects/ndfutil/venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tullettprebon.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Download fail, retry in 90 seconds...


<Response [404]>

In [30]:
from datetime import datetime

a = datetime.now()
a.date()

datetime.date(2023, 11, 15)

In [31]:
logger.remove()

In [32]:
d = ndf.download.download()
d.download_all()

bgc ok
tradition ok
prebontullet ok
gfi ok


In [33]:
mining = datamining()

In [34]:
mining.tulletprebon_calcs()

,Total for human,Volume
Class,,
BMF1,1.57B,1573700000
BMF2,1B,1000000000
BROKEN,50M,50000000
LONGER,184M,184000000
PTAX,78M,78000000
TOMPTAX,311M,311000000
TOTAL,3.37B,3370200000
XP,173.5M,173500000


In [35]:
mining.bgc_calcs()

,Total for human,Volume
Class,,
BMF,443.5M,443500000
BMF2,200M,200000000
BROKEN,120M,120000000
TOTAL,763.5M,763500000


In [36]:
mining.tradition_calcs()

,Total for human,Volume
Class,,
BROKEN,1.93B,1932200000
TOTAL,1.93B,1932200000


In [37]:
mining.gfi_calcs()

,Total for human,Volume
Class,,
BMF,648M,648000000
BMF2,50M,50000000
BROKEN,148M,148000000
TOTAL,846M,846000000


In [38]:
tulletprebon['source'] = 'TulletPrebon'
tradition['source'] = 'Tradition'
bgc['source'] = 'bgc'
gfi['source'] = 'gfi'

NameError: name 'tulletprebon' is not defined

In [ ]:
bgc

In [ ]:
df_result = pd.concat([tulletprebon, tradition, bgc, gfi])
df_result

In [ ]:
df_totals = df_result.query('Class == "TOTAL"')
df_totals

In [ ]:
df_totals.groupby(['Class']).sum()

In [ ]:
from numerize import numerize

market_total = df_totals.sum()['Volume']
market_total = numerize.numerize(float(market_total))
market_total